In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install transformers

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
tokenizer1 = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", use_fast = True)

In [ ]:
from transformers import AutoTokenizer
tokenizer2 = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast = True)

In [ ]:
model1 = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/test_bert_task1')
nlp1 = pipeline("sentiment-analysis", model = model1, tokenizer = tokenizer1)
model2 = AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/test_bert_ner2')
nlp2 = pipeline("token-classification", model = model2, tokenizer = tokenizer2)
model3 = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/test_bert_task3')
nlp3 = pipeline("sentiment-analysis", model = model3, tokenizer = tokenizer1)

In [ ]:
def post_process(tokens):
    post_process_text = ""
    for token in tokens:
        if not token.startswith("##"):
            post_process_text += " "

        post_process_text += token.lstrip("##")

    return post_process_text.lstrip()


# Output

In [ ]:
with open('/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/submission/output/dev_0001.txt', 'w') as output_file:
    with open('/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/submission/original public dataset/VLSP2023_ComOM_public_test_nolabel/dev_0001.txt', 'r') as input_file:
        lines = input_file.readlines()

    for line in lines:
        sentences = line.split('\t')
        if len(sentences) >= 2:
            output_file.write(sentences[0].strip() + '\n')

In [ ]:
comparative = []
task2 = []
input_file = '/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/submission/output/dev_0001.txt'
with open(input_file, 'r') as input_file:
  lines = input_file.readlines()
  for line in lines:
    comparative.append(nlp1(line))
for i, line in enumerate(lines):
    label = comparative[i][0]['label']
    if label == 'Comparative':
        task2.append(line)

In [ ]:
ner = []
for sentence in task2:
    ner.append(nlp2(sentence))

In [ ]:
preference = []
for sentence in task2:
    preference.append(nlp3(sentence))

In [ ]:
def post_process(tokens, entities):
    post_process_text = ""
    entity = []
    idx = 0
    for token in tokens:
        if not token.startswith("##"):
            post_process_text += " "
            entity.append(entities[idx])
        post_process_text += token.lstrip("##")
        idx += 1
    return post_process_text.lstrip(), entity

In [ ]:
import json
ner_post_processed = []
for token_list in ner:
    tokens = [token['word'] for token in token_list]
    entities = [token['entity'] for token in token_list]

    post_processed_text,entity = post_process(tokens,entities)
    post_processed_words = post_processed_text.split()

    result_list = [{'entity': entity[i], 'index': i + 1, 'word': post_processed_words[i]} for i in range(len(post_processed_words))]
    ner_post_processed.append(result_list)

In [ ]:
import json

json.dumps({'entity': 'LABEL_0', 'index': 14, 'word': 'và'}, ensure_ascii = False, indent=None)

'{"entity": "LABEL_0", "index": 14, "word": "và"}'

In [ ]:
input_file_path = '/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/submission/output/dev_0001.txt'
output_file_path = '/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/submission/output/dev_0001.txt'
task2 = []

with open(input_file_path, 'r') as input_file:
    lines = input_file.readlines()
idx = 0

with open(output_file_path, 'w') as output_file:
    for i, line in enumerate(lines):
        label = comparative[i][0]['label']
        if label == 'Comparative':
            task2.append(line)
            quintuple = {"subject": [], "object": [], "aspect": [], "predicate": [], "label": ""}
            for tag in ner_post_processed[idx]:
                entity = tag['entity']
                index = tag['index']
                word = tag['word']
                quintuple['label'] = preference[idx][0]['label']
                if entity in ['LABEL_1', 'LABEL_2']:
                    quintuple['subject'].append(f"{index}&&{word}")
                elif entity in ['LABEL_3', 'LABEL_4']:
                    quintuple['object'].append(f"{index}&&{word}")
                elif entity in ['LABEL_5', 'LABEL_6']:
                    quintuple['aspect'].append(f"{index}&&{word}")
                elif entity in ['LABEL_7', 'LABEL_8']:
                    quintuple['predicate'].append(f"{index}&&{word}")
            quintuple = json.dumps(quintuple, ensure_ascii = False)
            modified_line = f"{line.strip()}\n{quintuple}\n\n"
            idx += 1
        else:
            modified_line = f"{line.strip()}\n\n"
        output_file.write(modified_line)


In [ ]:
file_names = ['/content/drive/MyDrive/VLSP23/Data/HH/dev/dev_{:04d}_modified.txt'.format(i) for i in range(1, 25)]
# file_names = '/content/drive/MyDrive/VLSP23/Data/HH/dev/dev_0004_modified.txt'
output_file_path = '/content/drive/MyDrive/VLSP23/Data/HH/dev/output_count.txt'

sentence_count = 0
with open(output_file_path, 'w') as output_file:
  for file_name in file_names:
      with open(file_name, 'r') as input_file:
          lines = input_file.readlines()

      for line in lines:
          if line.strip() and not line.strip().startswith('{') and line != '\n':
              sentence_count += 1

              output_file.write(line)

print("Total sentences:", sentence_count)


Total sentences: 1732


In [ ]:
file_names = ['/content/drive/MyDrive/VLSP23/Data/HH/dev/dev_{:04d}.txt'.format(i) for i in range(1, 25)]
output_file_path = '/content/drive/MyDrive/VLSP23/Data/HH/dev/output_count.txt'

sentence_count = 0
with open(output_file_path, 'w') as output_file:
  for file_name in file_names:
      with open(file_name, 'r') as input_file:
          lines = input_file.readlines()

      for line in lines:
          sentences = line.split('\t')

          if len(sentences) >= 2 and not line.endswith("\t\xa0\n"):
              sentence_count += 1
              output_file.write(sentences[0].strip() + '\n')

print("Total sentences:", sentence_count)


Total sentences: 1732


In [ ]:
def post_process_for_ner(output, single=False):
    '''

    '''
    # Append to previous word
    result = []
    prev_entity = copy.copy(output[0])
    index = 1
    for entity in output[1:]:
        if "##" in entity["word"]:
            mod_word = entity["word"].replace("##", "")
            prev_entity["word"] += mod_word
            if prev_entity["entity"] != entity["entity"]:
                if single:
                    prev_entity["entity"] = copy.copy(prev_entity["entity"] if prev_entity["score"]>entity["score"] else entity["entity"])
                    prev_entity["score"] = copy.copy(max(prev_entity["score"], entity["score"]))
                else:
                    prev_entity["entity"] = copy.copy([prev_entity["entity"], entity["entity"]])
                    prev_entity["score"] = copy.copy([prev_entity["score"], entity["score"]])
        else:
            prev_entity["index"] = index
            index += 1
            result.append(copy.copy(prev_entity))
            prev_entity = entity

    result.append(prev_entity)
    return result